In [1]:
require 'nn'
require 'hdf5'
require 'optim'

### Loading the data:

In [76]:
myFile = hdf5.open('6-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [92]:
data

{
  valid_output : LongTensor - size: 3370x50
  train_1000 : LongTensor - size: 696825x7
  test : LongTensor - size: 3761x55
  nwords : LongTensor - size: 1
  train : LongTensor - size: 772670x7
  valid : LongTensor - size: 3370x55
}


### Creating the training set:

We need to remove the counts (used for the Count based language model)

In [81]:
nwords = 10001
train = data['train']:narrow(2,1,6)

In [82]:
train_input = train:narrow(2,1,5)
train_output = train:narrow(2,6,1)

In [89]:
N = 6
dwin = N-1
hid1 = 30.
hid2 = 100
nnlm = nn.Sequential()

tanh = nn.Sequential()
tanh:add(nn.LookupTable(nwords,hid1/dwin))
tanh:add(nn.View(1,-1,hid1))
tanh:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
tanh:add(nn.Linear(hid1,hid2))
tanh:add(nn.Tanh())

nnlm:add(tanh)
nnlm:add(nn.Linear(hid2, nwords))
nnlm:add(nn.LogSoftMax())

In [84]:
nnlm:forward(train_input:narrow(1,1,1)):size()

 10001
[torch.LongStorage of size 1]



In [90]:
criterion = nn.ClassNLLCriterion()

In [86]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train_input[i]:view(1,5), train_output[i]}
end
function dataset:size() return train:size(1) end -- 100 examples

In [91]:
--with regularisation:

sq_mat = torch.zeros(10001,6)
norm_mat = torch.zeros(10001,6)

for i = 1, 20 do
    
    timer = torch.Timer()
    if i ~= 1 then
        sq_mat:copy(torch.pow(nnlm:findModules('nn.LookupTable')[1].weight,2))
        norm_mat:copy(torch.expandAs(sq_mat:sum(2), sq_mat))
        nnlm:findModules('nn.LookupTable')[1].weight:cdiv(norm_mat)
    end
    
    trainer = nn.StochasticGradient(nnlm, criterion)
    trainer.learningRate = 0.01
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
end


# StochasticGradient: training	


# current error = 6.4245431634655	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.4245431634655	
Epoch 1: 1867.2106969357	


# StochasticGradient: training	


# current error = 5.9158270506484	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.9158270506484	


Epoch 2: 1851.8256881237	


...

# current error = 5.1854116802198	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.1854116802198	
Epoch 18: 1804.7857210636	


# StochasticGradient: training	


# current error = 5.1802477525104	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.1802477525104	
Epoch 19: 1804.3937339783	


# StochasticGradient: training	


# current error = 5.163963303637	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.163963303637	
Epoch 20: 1804.8741080761	


In [137]:
--save
torch.save('nnlm1',nnlm)

In [75]:
-- Without regularisation:

trainer = nn.StochasticGradient(nnlm, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 5
trainer:train(dataset)

# StochasticGradient: training	


# current error = 4.3886357784844	


# current error = 4.1514940863395	


# current error = 4.0717284582465	


# current error = 4.0314867838883	


# current error = 4.00779912249	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 4.00779912249	


### Tentative de Validation:

In [212]:
--load:
nnlm = torch.load('../../nnlm1')

In [244]:
kag = data['valid_output']

In [246]:
kag = data['valid_output']
valid = data['valid']
valid_topredict = valid:narrow(2,1,50)
valid_input = valid:narrow(2,51,5)
preds_valid = nnlm:forward(valid_input)
kag_pred_valid = torch.Tensor(preds_valid:size(1),50)
for i = 1,preds_valid:size(1) do
    kag_pred_valid[i]:copy(preds_valid[i]:index(1,valid_topredict[i])):exp()
end
norm_mat_ = torch.zeros(preds_valid:size(1),50)
norm_mat_:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
kag_pred_valid:cdiv(norm_mat_)
perp = 0
for i = 1,preds_valid:size(1) do
    mm,aa = kag[i]:max(1)
    perp = perp + math.log(kag_pred_valid[i][aa[1]])
end
perp = math.exp(-perp/preds_valid:size(1))
print(perp)

In [248]:
preds_valid = nnlm:forward(valid_input)

In [249]:
kag_pred_valid = torch.Tensor(preds_valid:size(1),50)
for i = 1,preds_valid:size(1) do
    kag_pred_valid[i]:copy(preds_valid[i]:index(1,valid_topredict[i])):exp()
end

In [250]:
norm_mat_ = torch.zeros(preds_valid:size(1),50)
norm_mat_:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
kag_pred_valid:cdiv(norm_mat_)

In [251]:
acc = 0
for i = 1,preds_valid:size(1) do
    m,a = kag_pred_valid[i]:max(1)
    mm,aa = kag[i]:max(1)
    if aa[1] == a[1] then
        acc = acc + 1
    end
end
print(acc/3370.)

0.42581602373887	


In [138]:
function perplexity(distribution)
    local perp = torch.DoubleTensor(distribution:size(1), distribution:size(2))
    perp:copy(distribution)
    n = distribution:size(1) * distribution:size(2)
    return math.exp(-(1/n)* perp:log():sum())
end

In [252]:
perp = 0
for i = 1,preds_valid:size(1) do
    mm,aa = kag[i]:max(1)
    perp = perp + math.log(kag_pred_valid[i][aa[1]])
end
perp = math.exp(-perp/preds_valid:size(1))
print(perp)

26.309662260245	


In [253]:
kag_pred_valid[preds_valid:size(1)][aa[1]]

0.49076318656613	


In [156]:
dataset[1]

{
  1 : LongTensor - size: 1x5
  2 : LongTensor - size: 1
}


### Deep:

In [206]:
N = 6
dwin = N-1
hid1 = 30.
hid2 = 100
dnnlm = nn.Sequential()

LT = nn.Sequential()
LT:add(nn.LookupTable(nwords,hid1/dwin))
LT:add(nn.View(1,-1,hid1))
LT:add(nn.Squeeze()) 

dnnlm:add(LT)

concat = nn.ConcatTable()

lin_tanh = nn.Sequential()
lin_tanh:add(nn.Linear(hid1,hid2))
lin_tanh:add(nn.Tanh())

id = nn.Identity()

concat:add(lin_tanh)
concat:add(id)

dnnlm:add(concat)
dnnlm:add(nn.JoinTable(1))
dnnlm:add(nn.Linear(hid2+hid1, nwords))
dnnlm:add(nn.LogSoftMax())

In [207]:
t = train_input[8000]

In [208]:
tt = dnnlm:forward(t)

In [189]:
nn.JoinTable(1):forward(ttt):size()

 130
[torch.LongStorage of size 1]



In [ ]:
criterion = nn.ClassNLLCriterion()

In [272]:
sq_mat:sum(2):max()

38.705445706632	


In [214]:
--with regularisation:

sq_mat = torch.zeros(10001,6)
norm_mat = torch.zeros(10001,6)


for i = 2, 15 do
    
    timer = torch.Timer()
    if i ~= 1 then
        sq_mat:copy(torch.pow(dnnlm:findModules('nn.LookupTable')[1].weight,2))
        norm_mat:copy(torch.expandAs(sq_mat:sum(2), sq_mat))
        nnlm:findModules('nn.LookupTable')[1].weight:cdiv(norm_mat)
    end
    
    trainer = nn.StochasticGradient(dnnlm, criterion)
    trainer.learningRate = 0.01
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
    
end

# StochasticGradient: training	


# current error = 6.0753078968396	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.0753078968396	
Epoch 2: 2378.5694739819	


# StochasticGradient: training	


# current error = 5.9075009795102	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.9075009795102	
Epoch 3: 2383.4352579117	


# StochasticGradient: training	


...

# current error = 5.4319642996491	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.4319642996491	
Epoch 13: 4948.130671978	


# StochasticGradient: training	


# current error = 5.4142337359248	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.4142337359248	
Epoch 14: 2365.0248579979	


# StochasticGradient: training	


# current error = 5.3984644474863	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.3984644474863	
Epoch 15: 2369.2235422134	


In [254]:
kag_pred_valid = torch.Tensor(preds_valid:size(1),50)
for i = 1,preds_valid:size(1) do
    kag_pred_valid[i]:copy(dnnlm:forward(valid_input[i]):index(1,valid_topredict[i])):exp()
end

In [255]:
norm_mat_ = torch.zeros(preds_valid:size(1),50)
norm_mat_:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
kag_pred_valid:cdiv(norm_mat_)

In [228]:
acc = 0
for i = 1,preds_valid:size(1) do
    m,a = kag_pred_valid[i]:max(1)
    mm,aa = kag[i]:max(1)
    if aa[1] == a[1] then
        acc = acc + 1
    end
end
print(acc/3370.)

0.57833827893175	


In [258]:
CE = 0
for i = 1,preds_valid:size(1) do
    mm,aa = kag[i]:max(1)
    CE = CE + math.log(kag_pred_valid[i][aa[1]])
end

In [259]:
print('Perplexity is equal to: '.. math.exp(-CE/preds_valid:size(1)))

Perplexity is equal to: 7.3734290599113	


### Testing on test data:

In [238]:
test_data = data['test']
test_topredict = test_data:narrow(2,1,50)
test_input = test_data:narrow(2,51,5)


In [241]:
kag_pred_test = torch.Tensor(test_data:size(1),50)
for i = 1,test_data:size(1) do
    kag_pred_test[i]:copy(dnnlm:forward(test_input[i]):index(1,test_topredict[i])):exp()
end

In [242]:
norm_mat_ = torch.zeros(test_data:size(1),50)
norm_mat_:copy(torch.expandAs(kag_pred_test:sum(2), kag_pred_test))
kag_pred_test:cdiv(norm_mat_)

In [243]:
filename = 'tocsv_1.f5'
myFile = hdf5.open(filename, 'w')
myFile:write(filename, kag_pred_test)
myFile:close()  

### Trying with 4-grams and changing embeddings size:

In [2]:
myFile = hdf5.open('../../5-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [3]:
train5 = data['train_nocounts']
train_input5 = train5:narrow(2,1,4)
train_output5 = train5:narrow(2,5,1)

In [11]:
dataset5={};
for i=1,train5:size(1) do 
  dataset5[i] = {train_input5[i]:view(1,4), train_output5[i]}
end
function dataset5:size() return train5:size(1) end -- 100 examples

In [13]:
kag5 = data['valid_output']
valid5 = data['valid']
valid_topredict5 = valid5:narrow(2,1,50)
valid_input5 = valid5:narrow(2,51,4)

In [42]:
nwords = 10001
N = 5
dwin = N-1
hid1 = 30.
hid2 = 80
dnnlm2 = nn.Sequential()

LT2 = nn.Sequential()
LT1 = nn.LookupTable(nwords,hid1)
LT2:add(LT1)
LT2:add(nn.View(1,-1,hid1*dwin))
LT2:add(nn.Squeeze()) 

dnnlm2:add(LT2)

concat2 = nn.ConcatTable()

lin_tanh2 = nn.Sequential()
lin_tanh2:add(nn.Linear(hid1*dwin,hid2))
lin_tanh2:add(nn.Tanh())

id2 = nn.Identity()

concat2:add(lin_tanh2)
concat2:add(id2)

dnnlm2:add(concat2)
dnnlm2:add(nn.JoinTable(1))
dnnlm2:add(nn.Linear(hid2+hid1*dwin, nwords))
dnnlm2:add(nn.LogSoftMax())

In [25]:
criterion = nn.ClassNLLCriterion()

In [26]:
kag_pred_valid5 = torch.Tensor(valid_input5:size(1),50)
norm_mat_5 = torch.zeros(valid_input5:size(1),50)
sq_mat = torch.zeros(10001,30)

val_res = torch.Tensor(10)

In [ ]:
--with regularisation:

for i = 1, 1 do
    
    timer = torch.Timer()
    
    LT1.weight:renorm(2,2,1)
    
    trainer2 = nn.StochasticGradient(dnnlm2, criterion)
    trainer2.learningRate = 0.005
    trainer2.maxIteration = 1
    trainer2:train(dataset5)
    print('Epoch '..i..': '..timer:time().real)
    
    kag_pred_valid5:zero()

    for i = 1,valid5:size(1) do
        kag_pred_valid5[i]:copy(dnnlm2:forward(valid_input5[i]):index(1,valid_topredict5[i])):exp()
    end
    
    norm_mat_5:zero()
    norm_mat_5:copy(torch.expandAs(kag_pred_valid5:sum(2), kag_pred_valid5))
    kag_pred_valid5:cdiv(norm_mat_5)
    
    CE = 0
    for i = 1,kag_pred_valid5:size(1) do
        mm,aa = kag5[i]:max(1)
        CE = CE + math.log(kag_pred_valid5[i][aa[1]])
    end
    
    val_res[i] = math.exp(-CE/kag_pred_valid5:size(1))
    print('Perplexity on valid: '..val_res[i])
    
end

# StochasticGradient: training	


 0.7071  0.7071
 0.7071  0.7071
[torch.DoubleTensor of size 2x2]



### Using Optim

In [2]:
myFile = hdf5.open('../../5-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [3]:
data

{
  train_nocounts : DoubleTensor - size: 887522x5
  train_1000 : DoubleTensor - size: 887522x5
  test : LongTensor - size: 3761x54
  train : LongTensor - size: 754037x6
  nwords : LongTensor - size: 1
  valid : LongTensor - size: 3370x54
  train_1000_nocounts : LongTensor - size: 620208x6
  valid_output : LongTensor - size: 3370x50
}


In [4]:
train5 = data['train_nocounts']
train_input5 = train5:narrow(2,1,4)
train_output5 = train5:narrow(2,5,1)

In [5]:
train_output5:max()

10001	


In [6]:
criterion = nn.ClassNLLCriterion()

In [7]:
nwords = 10001
N = 5
dwin = N-1
hid1 = 30.
hid2 = 80
dnnlm2 = nn.Sequential()

LT2 = nn.Sequential()
LT1 = nn.LookupTable(nwords,hid1)
LT2:add(LT1)
LT2:add(nn.View(1,-1,hid1*dwin))
LT2:add(nn.Squeeze()) 

dnnlm2:add(LT2)

concat2 = nn.ConcatTable()

lin_tanh2 = nn.Sequential()
lin_tanh2:add(nn.Linear(hid1*dwin,hid2))
lin_tanh2:add(nn.Tanh())

id2 = nn.Identity()

concat2:add(lin_tanh2)
concat2:add(id2)

dnnlm2:add(concat2)
dnnlm2:add(nn.JoinTable(2))
dnnlm2:add(nn.Linear(hid2+hid1*dwin, nwords))
dnnlm2:add(nn.LogSoftMax())

In [8]:
model = dnnlm2

In [9]:
param, gradparam = model:getParameters()
batchsize = 64

In [16]:
for e = 1,5 do
    timer = torch.Timer()
    avLoss = 0
    
    LT1.weight:renorm(2,2,1)
    
    for t = 1,train5:size(1),batchsize do
    -- for t = 1,1,1 do
        inputs = torch.zeros(math.min(batchsize,train5:size(1)-t+1),dwin)
        targets = torch.zeros(math.min(batchsize,train5:size(1)-t+1))
        k = 1

        for i = t,math.min(t+batchsize-1,train5:size(1)) do
            targets[k] = train_output5[i]
            inputs[k] = train_input5[i]
            k = k + 1
        end

        function feval(x)
            collectgarbage()

            if x ~= param then
                param:copy(x)
            end

            gradparam:zero()

            outputs = model:forward(inputs)
            L = criterion:forward(outputs,targets)
            dL = criterion:backward(outputs,targets)
            model:backward(inputs,dL)

            avLoss = avLoss + L

            return L,gradparam

        end

        sgdState = {learningRate = 0.005}

        optim.sgd(feval,param,sgdState)

    end
    print('Epoch '.. e ..'completed in (sec): '..timer:time().real)
    print('Average Loss on train: '.. avLoss/math.floor(train5:size(1)/batchsize))
end

Epoch 1completed in (sec): 575.05744695663	
Average Loss on train: 7.922858046091	


Epoch 2completed in (sec): 559.13870000839	
Average Loss on train: 6.9411545443104	


Epoch 3completed in (sec): 554.5067191124	
Average Loss on train: 6.7439821096093	


Epoch 4completed in (sec): 549.67982292175	
Average Loss on train: 6.6323669364111	


Epoch 5completed in (sec): 554.31065011024	
Average Loss on train: 6.5619067452224	


In [17]:
for e = 6,20 do
    timer = torch.Timer()
    avLoss = 0
    
    LT1.weight:renorm(2,2,1)
    
    for t = 1,train5:size(1),batchsize do
    -- for t = 1,1,1 do
        inputs = torch.zeros(math.min(batchsize,train5:size(1)-t+1),dwin)
        targets = torch.zeros(math.min(batchsize,train5:size(1)-t+1))
        k = 1

        for i = t,math.min(t+batchsize-1,train5:size(1)) do
            targets[k] = train_output5[i]
            inputs[k] = train_input5[i]
            k = k + 1
        end

        function feval(x)
            collectgarbage()

            if x ~= param then
                param:copy(x)
            end

            gradparam:zero()

            outputs = model:forward(inputs)
            L = criterion:forward(outputs,targets)
            dL = criterion:backward(outputs,targets)
            model:backward(inputs,dL)

            avLoss = avLoss + L

            return L,gradparam

        end

        sgdState = {learningRate = 0.005}

        optim.sgd(feval,param,sgdState)

    end
    print('Epoch '.. e ..'completed in (sec): '..timer:time().real)
    print('Average Loss on train: '.. avLoss/math.floor(train5:size(1)/batchsize))
end

Epoch 6completed in (sec): 549.36122202873	
Average Loss on train: 6.5131466153525	


Epoch 7completed in (sec): 551.05811500549	


Average Loss on train: 6.4747375418067	


Epoch 8completed in (sec): 550.92656683922	
Average Loss on train: 6.4417819511198	


Epoch 9completed in (sec): 551.57784795761	
Average Loss on train: 6.4115803367962	


...

Epoch 16completed in (sec): 545.59943294525	
Average Loss on train: 6.2394098589757	


Epoch 17completed in (sec): 545.86253595352	
Average Loss on train: 6.2199206628517	


Epoch 18completed in (sec): 546.13499903679	
Average Loss on train: 6.2014229691107	


Epoch 19completed in (sec): 545.99924588203	
Average Loss on train: 6.1838587273178	


Epoch 20completed in (sec): 545.85558891296	
Average Loss on train: 6.1671739215651	


In [19]:
kag = data['valid_output']
valid = data['valid']

In [21]:
valid_topredict = valid:narrow(2,1,50)
valid_input = valid:narrow(2,51,4)
preds_valid = model:forward(valid_input)

In [22]:
preds_valid = model:forward(valid_input)
kag_pred_valid = torch.Tensor(preds_valid:size(1),50)
for i = 1,preds_valid:size(1) do
    kag_pred_valid[i]:copy(preds_valid[i]:index(1,valid_topredict[i])):exp()
end
norm_mat_ = torch.zeros(preds_valid:size(1),50)
norm_mat_:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
kag_pred_valid:cdiv(norm_mat_)
perp = 0
for i = 1,preds_valid:size(1) do
    mm,aa = kag[i]:max(1)
    perp = perp + math.log(kag_pred_valid[i][aa[1]])
end
perp = math.exp(-perp/preds_valid:size(1))
print(perp)

7.2295951783231	


### 6-gram & optim:

In [2]:
myFile = hdf5.open('../../6-grams.hdf5','r')
data6 = myFile:all()
myFile:close()

train6 = data6['train_nocounts']
train_input6 = train6:narrow(2,1,5)
train_output6 = train6:narrow(2,5,1)

kag = data6['valid_output']
valid = data6['valid']

valid_topredict = valid:narrow(2,1,50)
valid_input = valid:narrow(2,51,5)

criterion = nn.ClassNLLCriterion()

nwords = 10001
N = 6
dwin = N-1
hid1 = 30.
hid2 = 100
dnnlm3 = nn.Sequential()

LT2_3 = nn.Sequential()
LT1_3 = nn.LookupTable(nwords,hid1)
LT2_3:add(LT1_3)
LT2_3:add(nn.View(1,-1,hid1*dwin))
LT2_3:add(nn.Squeeze()) 

dnnlm3:add(LT2_3)

concat2_3 = nn.ConcatTable()

lin_tanh2_3 = nn.Sequential()
lin_tanh2_3:add(nn.Linear(hid1*dwin,hid2))
lin_tanh2_3:add(nn.Tanh())

id2_3 = nn.Identity()

concat2_3:add(lin_tanh2_3)
concat2_3:add(id2_3)

dnnlm3:add(concat2_3)
dnnlm3:add(nn.JoinTable(2))
dnnlm3:add(nn.Linear(hid2+hid1*dwin, nwords))
dnnlm3:add(nn.LogSoftMax())

In [3]:
model2 = dnnlm3
param2, gradparam2 = model2:getParameters()
batchsize = 32

In [4]:
for e = 1,20 do
    timer = torch.Timer()
    avLoss = 0
    
    LT1_3.weight:renorm(2,2,1)
    
    for t = 1,train6:size(1),batchsize do
    -- for t = 1,1,1 do
        inputs = torch.zeros(math.min(batchsize,train6:size(1)-t+1),dwin)
        targets = torch.zeros(math.min(batchsize,train6:size(1)-t+1))
        k = 1

        for i = t,math.min(t+batchsize-1,train6:size(1)) do
            targets[k] = train_output6[i]
            inputs[k] = train_input6[i]
            k = k + 1
        end

        function feval(x)
            collectgarbage()

            if x ~= param2 then
                param2:copy(x)
            end

            gradparam2:zero()

            outputs = model2:forward(inputs)
            L = criterion:forward(outputs,targets)
            dL = criterion:backward(outputs,targets)
            model2:backward(inputs,dL)

            avLoss = avLoss + L

            return L,gradparam2

        end

        sgdState = {learningRate = 0.01}

        optim.sgd(feval,param2,sgdState)

    end
    print('Epoch '.. e ..'completed in (sec): '..timer:time().real)
    print('Average Loss on train: '.. avLoss/math.floor(train6:size(1)/batchsize))
    
--     preds_valid = model2:forward(valid_input)
--     kag_pred_valid = torch.Tensor(preds_valid:size(1),50)
--     for i = 1,preds_valid:size(1) do
--         kag_pred_valid[i]:copy(preds_valid[i]:index(1,valid_topredict[i])):exp()
--     end
--     norm_mat_ = torch.zeros(preds_valid:size(1),50)
--     norm_mat_:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
--     kag_pred_valid:cdiv(norm_mat_)
--     perp = 0
--     for i = 1,preds_valid:size(1) do
--         mm,aa = kag[i]:max(1)
--         perp = perp + math.log(kag_pred_valid[i][aa[1]])
--     end
--     perp = math.exp(-perp/preds_valid:size(1))
--     print('Perpelixty on validation: '..perp)
    
end

Epoch 1completed in (sec): 878.22259616852	
Average Loss on train: 5.6776661696923	


Epoch 2completed in (sec): 867.09469199181	
Average Loss on train: 4.4994644587894	


Epoch 3completed in (sec): 862.59846186638	
Average Loss on train: 4.1251513314738	


Epoch 4completed in (sec): 863.19748187065	
Average Loss on train: 3.9031739403228	


Epoch 5completed in (sec): 860.86750912666	
Average Loss on train: 3.726930405877	


...

Epoch 16completed in (sec): 857.98085784912	
Average Loss on train: 2.5498510691477	


Epoch 17completed in (sec): 859.30982708931	
Average Loss on train: 2.4836568318137	


Epoch 18completed in (sec): 857.0586810112	
Average Loss on train: 2.4217574490704	


Epoch 19completed in (sec): 856.88945007324	
Average Loss on train: 2.3637138241296	


Epoch 20completed in (sec): 857.33166599274	
Average Loss on train: 2.3091430400983	


Validation:

In [84]:
pred_valid = model2:forward(valid_input)

In [85]:
kag_pred_valid = torch.Tensor(pred_valid:size(1))

In [86]:
for i = 1,3370 do
    mm,aa = kag[i]:max(1)
    kag_pred_valid[i] = valid_topredict[i][aa[1]]
end


In [87]:
L,n = criterion:forward(pred_valid,kag_pred_valid)

In [95]:
math.exp(L/n)

1.0052986466555	


In [82]:
data6

{
  train_nocounts : DoubleTensor - size: 887522x6
  train_1000 : DoubleTensor - size: 887522x6
  test : LongTensor - size: 3761x55
  train : LongTensor - size: 772670x7
  nwords : LongTensor - size: 1
  valid : LongTensor - size: 3370x55
  train_1000_nocounts : LongTensor - size: 696825x7
  valid_output : LongTensor - size: 3370x50
}


In [91]:
testd = data6['test']

test_topredict = testd:narrow(2,1,50)
test_input = testd:narrow(2,51,5)

In [96]:
pred_test = model2:forward(test_input)

In [98]:
kag_pred_test = torch.Tensor(pred_test:size(1),50)
for i = 1,pred_test:size(1) do
    kag_pred_test[i]:copy(pred_test[i]:index(1,test_topredict[i])):exp()
end

In [101]:
kag_pred_test:renorm(1,1,1)

75.281482807739	


In [102]:
filename = 'tocsv_.f5'
myFile = hdf5.open(filename, 'w')
myFile:write(filename, kag_pred_test)
myFile:close()  

In [94]:
torch.save('model2',model2)

#### Other normalisation:

In [5]:
kag = data6['valid_output']
valid = data6['valid']

valid_topredict = valid:narrow(2,1,50)
valid_input = valid:narrow(2,51,5)

criterion = nn.ClassNLLCriterion()

nwords = 10001
N = 6
dwin = N-1
hid1 = 30.
hid2 = 100
dnnlm4 = nn.Sequential()

LT2_4 = nn.Sequential()
LT1_4 = nn.LookupTable(nwords,hid1)
LT2_4:add(LT1_4)
LT2_4:add(nn.View(1,-1,hid1*dwin))
LT2_4:add(nn.Squeeze()) 

dnnlm4:add(LT2_4)

concat2_4 = nn.ConcatTable()

lin_tanh2_4 = nn.Sequential()
lin_tanh2_4:add(nn.Linear(hid1*dwin,hid2))
lin_tanh2_4:add(nn.Tanh())

id2_4 = nn.Identity()

concat2_4:add(lin_tanh2_4)
concat2_4:add(id2_4)

dnnlm4:add(concat2_4)
dnnlm4:add(nn.JoinTable(2))
dnnlm4:add(nn.Linear(hid2+hid1*dwin, nwords))
dnnlm4:add(nn.LogSoftMax())

In [6]:
model3 = dnnlm4
param3, gradparam3 = model3:getParameters()
batchsize = 32

In [7]:
for e = 1,20 do
    timer = torch.Timer()
    avLoss = 0
    
    LT1_4.weight:renorm(2,2,1)
    
    for t = 1,train6:size(1),batchsize do
    -- for t = 1,1,1 do
        inputs = torch.zeros(math.min(batchsize,train6:size(1)-t+1),dwin)
        targets = torch.zeros(math.min(batchsize,train6:size(1)-t+1))
        k = 1

        for i = t,math.min(t+batchsize-1,train6:size(1)) do
            targets[k] = train_output6[i]
            inputs[k] = train_input6[i]
            k = k + 1
        end

        function feval(x)
            collectgarbage()

            if x ~= param3 then
                param3:copy(x)
            end

            gradparam3:zero()

            outputs = model3:forward(inputs)
            L = criterion:forward(outputs,targets)
            dL = criterion:backward(outputs,targets)
            model3:backward(inputs,dL)

            avLoss = avLoss + L

            return L,gradparam3

        end

        sgdState = {learningRate = 0.01}

        optim.sgd(feval,param3,sgdState)

    end
    print('Epoch '.. e ..'completed in (sec): '..timer:time().real)
    print('Average Loss on train: '.. avLoss/math.floor(train6:size(1)/batchsize))
    
--     preds_valid = model3:forward(valid_input)
--     kag_pred_valid = torch.Tensor(preds_valid:size(1),50)
--     for i = 1,preds_valid:size(1) do
--         kag_pred_valid[i]:copy(preds_valid[i]:index(1,valid_topredict[i])):exp()
--     end
--     norm_mat_ = torch.zeros(preds_valid:size(1),50)
--     norm_mat_:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
--     kag_pred_valid:cdiv(norm_mat_)
--     perp = 0
--     for i = 1,preds_valid:size(1) do
--         mm,aa = kag[i]:max(1)
--         perp = perp + math.log(kag_pred_valid[i][aa[1]])
--     end
--     perp = math.exp(-perp/preds_valid:size(1))
--     print('Perpelixty on validation: '..perp)
    
end

Epoch 1completed in (sec): 847.72674679756	
Average Loss on train: 5.6841756605162	


Epoch 2completed in (sec): 841.65930008888	
Average Loss on train: 4.4876136760813	


Epoch 3completed in (sec): 842.08484983444	
Average Loss on train: 4.117381326827	


Epoch 4completed in (sec): 843.27526092529	
Average Loss on train: 3.9026942939606	


Epoch 5completed in (sec): 842.78929591179	
Average Loss on train: 3.7258989579408	


...

Epoch 16completed in (sec): 845.52367210388	
Average Loss on train: 2.5578975305521	


Epoch 17completed in (sec): 844.53724098206	
Average Loss on train: 2.4922512518373	


Epoch 18completed in (sec): 848.8222618103	
Average Loss on train: 2.4308572871654	


Epoch 19completed in (sec): 855.49179291725	
Average Loss on train: 2.3732306759738	


Epoch 20completed in (sec): 855.15243601799	
Average Loss on train: 2.318980925103	


In [15]:
A = torch.Tensor({{1,2}})
B = torch.Tensor(2)
B:copy(A)
print B:pow(2)
print A

[string "A = torch.Tensor({{1,2}})..."]:4: '=' expected near 'B': 

In [23]:
A = torch.Tensor({{1,2},{3,3}})

In [24]:
A:renorm(1,1,1)

 0.3333  0.6667
 0.5000  0.5000
[torch.DoubleTensor of size 2x2]



In [18]:
B:copy(A)



 1
 2
[torch.DoubleTensor of size 2]



In [19]:
B:pow(2)

 1
 4
[torch.DoubleTensor of size 2]



-2.4716e-04
-4.1371e+01
-3.1167e+01
-2.9833e+01
-2.3927e+01
-2.5882e+01
-2.7556e+01
-1.5502e+01
-1.6047e+01
-1.5411e+01
-2.1350e+01
-1.5061e+01
-1.8774e+01
-1.8910e+01
-2.0084e+01
-2.1533e+01
-3.5448e+01
-2.2587e+01
-1.5116e+01
-2.3005e+01
-2.3868e+01
-1.0768e+01
-2.1217e+01
-1.2884e+01
-1.4675e+01
-1.3561e+01
-1.5709e+01
-4.1916e+01
-2.2500e+01
-2.0729e+01
-1.7907e+01
-2.8915e+01
-1.7927e+01
-3.1793e+01
-2.4770e+01
-2.8043e+01
-2.0006e+01
-2.9388e+01
-2.7062e+01
-1.8164e+01
-2.3852e+01
-1.1959e+01
-1.9698e+01
-3.1137e+01
-1.2483e+01
-2.6343e+01
-2.2074e+01
-3.5267e+01
-2.5712e+01
-2.0593e+01
-9.6548e+00
-3.1754e+01
-2.2867e+01
-2.8163e+01
-2.1382e+01
-1.1752e+01
-2.2015e+01
-2.6996e+01
-1.9057e+01
-1.7552e+01
-1.8750e+01
-3.2011e+01
-1.1373e+01
-2.1230e+01
-1.4336e+01
-3.4074e+01
-3.3370e+01
-2.0883e+01
-2.3256e+01
-2.4328e+01
-1.8873e+01
-1.8203e+01
-1.7375e+01
-3.6748e+01
-2.5088e+01
-2.7900e+01
-1.4358e+01
-3.3918e+01
-1.7196e+01
-1.0580e+01
-3.2333e+01
-2.7790e+01
-2.2680e+01
-2.9

407e+01
-1.5728e+01
-3.6949e+01
-3.3608e+01
-4.4842e+01
-3.6255e+01
-2.8810e+01
-3.2464e+01
-3.4668e+01
-2.3273e+01
-3.5895e+01
-3.5381e+01
-2.4850e+01
-3.7666e+01
-3.4122e+01
-4.0664e+01
-3.3417e+01
-3.8004e+01
-4.1178e+01
-3.9323e+01
-1.6707e+01
-2.7885e+01
-3.4359e+01
-3.0813e+01
-3.7919e+01
-3.6917e+01
-3.0036e+01
-2.0615e+01
-2.5298e+01
-1.9371e+01
-3.2662e+01
-2.9475e+01
-3.9861e+01
-2.3607e+01
-3.4077e+01
-2.2881e+01
-2.6034e+01
-3.1383e+01
-2.3706e+01
-4.0616e+01
-2.2879e+01
-3.2004e+01
-2.4539e+01
-2.6922e+01
-2.4846e+01
-3.3802e+01
-2.8815e+01
-2.0695e+01
-3.8340e+01
-1.7593e+01
-2.6853e+01
-2.6416e+01
-3.6026e+01
-3.3503e+01
-2.1741e+01
-3.1319e+01
-3.0736e+01
-3.0390e+01
-3.3928e+01
-2.2394e+01
-3.3529e+01
-3.1358e+01
-3.5780e+01
-3.1598e+01
-2.9639e+01
-1.8408e+01
-3.5501e+01
-1.8272e+01
-3.7079e+01
-3.4101e+01
-3.9385e+01
-1.9607e+01
-3.3805e+01
-2.2222e+01
-3.0240e+01
-2.2157e+01
-3.5428e+01
-3.7841e+01
-3.0572e+01
-3.3032e+01
-2.7228e+01
-3.7296e+01
-3.7214e+01
-2.1615e

+01
-2.5069e+01
-2.9925e+01
-3.0926e+01
-2.6312e+01
-2.7946e+01
-2.9277e+01
-3.2689e+01
-3.1877e+01
-2.6700e+01
-2.8650e+01
-2.4115e+01
-3.2836e+01
-2.7919e+01
-3.0374e+01
-2.9263e+01
-3.3354e+01
-3.0076e+01
-2.8026e+01
-2.5743e+01
-2.8331e+01
-2.7200e+01
-3.4719e+01
-3.3020e+01
-2.6386e+01
-2.3639e+01
-2.9191e+01
-2.9185e+01
-2.2300e+01
-2.6593e+01
-2.4158e+01
-2.2312e+01
-3.1893e+01
-3.1139e+01
-2.4912e+01
-3.3547e+01
-3.0470e+01
-3.1515e+01
-2.8794e+01
-3.1523e+01
-3.2318e+01
-3.1407e+01
-3.1432e+01
-2.5947e+01
-2.9729e+01
-2.4153e+01
-3.0374e+01
-2.4945e+01
-2.8234e+01
-3.2048e+01
-2.6492e+01
-3.4717e+01
-2.5119e+01
-3.1811e+01
-2.5940e+01
-2.9001e+01
-3.0301e+01
-2.9217e+01
-2.9818e+01
-2.5429e+01
-3.0949e+01
-2.5697e+01
-2.2949e+01
-2.9725e+01
-3.0663e+01
-2.5506e+01
-2.6194e+01
-2.3354e+01
-3.2013e+01
-3.2500e+01
-3.2769e+01
-2.9627e+01
-2.4324e+01
-3.2146e+01
-3.0598e+01
-2.9712e+01
-3.0714e+01
-3.3482e+01
-3.3407e+01
-3.0400e+01
-3.1036e+01
-2.6534e+01
-2.5608e+01
-2.3792e+01


-2.7781e+01
-2.8015e+01
-2.6568e+01
-2.7121e+01
-2.7066e+01
-2.9276e+01
-2.7608e+01
-2.7299e+01
-2.6722e+01
-2.7146e+01
-3.0841e+01
-2.7658e+01
-2.8159e+01
-3.0577e+01
-2.7572e+01
-2.9336e+01
-3.0737e+01
-2.8518e+01
-3.0812e+01
-2.8139e+01
-3.0053e+01
-3.1031e+01
-3.1450e+01
-2.7480e+01
-2.8599e+01
-3.1277e+01
-2.6842e+01
-2.8786e+01
-2.8114e+01
-2.9460e+01
-3.0820e+01
-2.9135e+01
-2.8608e+01
-2.7836e+01
-2.6683e+01
-2.8450e+01
-2.8160e+01
-2.8888e+01
-2.9918e+01
-2.7844e+01
-3.1537e+01
-2.9234e+01
-3.0915e+01
-2.8561e+01
-2.8402e+01
-2.8660e+01
-3.0090e+01
-2.6180e+01
-3.1701e+01
-2.7314e+01
-2.9686e+01
-2.8335e+01
-2.9591e+01
-3.0719e+01
-2.9234e+01
-3.0892e+01
-2.9558e+01
-3.0135e+01
-3.0572e+01
-2.7475e+01
-2.9183e+01
-2.5521e+01
-2.8736e+01
-2.9598e+01
-2.9080e+01
-2.9845e+01
-2.7279e+01
-2.8084e+01
-2.9049e+01
-2.8723e+01
-3.0847e+01
-2.8011e+01
-2.8994e+01
-2.7983e+01
-2.7707e+01
-2.8207e+01
-2.7691e+01
-2.7846e+01
-2.8313e+01
-2.7349e+01
-2.6898e+01
-2.8658e+01
-2.8447e+01
-3.0

262e+01
-2.8386e+01
-3.0007e+01
-3.0543e+01
-2.8721e+01
-2.6894e+01
-2.7282e+01
-2.9556e+01
-2.9748e+01
-2.7332e+01
-2.8465e+01
-2.7510e+01
-2.9054e+01
-2.6778e+01
-2.7985e+01
-2.7823e+01
-2.8036e+01
-2.8615e+01
-2.9349e+01
-2.7014e+01
-2.8997e+01
-2.8867e+01
-2.8122e+01
-2.8809e+01
-2.9456e+01
-2.8271e+01
-2.9129e+01
-2.7449e+01
-2.8962e+01
-2.8152e+01
-2.8548e+01
-2.9986e+01
-2.8120e+01
-2.8978e+01
-2.7387e+01
-2.8953e+01
-2.9740e+01
-2.7836e+01
-2.8904e+01
-2.8419e+01
-2.8957e+01
-2.8788e+01
-2.9780e+01
-2.8156e+01
-2.8891e+01
-2.7384e+01
-2.9485e+01
-2.8675e+01
-2.9188e+01
-2.7833e+01
-3.0366e+01
-2.8722e+01
-2.8771e+01
-2.9478e+01
-2.8279e+01
-2.8164e+01
-2.6489e+01
-3.0068e+01
-2.8488e+01
-2.8813e+01
-2.7853e+01
-2.8880e+01
-2.8266e+01
-2.9260e+01
-2.9779e+01
-2.7830e+01
-2.7027e+01
-2.8081e+01
-2.8438e+01
-2.8035e+01
-2.7158e+01
-2.9074e+01
-2.9227e+01
-2.9066e+01
-2.8555e+01
-2.8047e+01
-2.8838e+01
-2.9175e+01
-2.9164e+01
-2.9183e+01
-2.8020e+01
-3.0675e+01
-2.9748e+01
-2.8021e

...

677e+01
-2.9597e+01
-3.0512e+01
-2.9737e+01
-2.9523e+01
-2.9990e+01
-2.9784e+01
-2.9843e+01
-2.9773e+01
-3.0194e+01
-3.0156e+01
-2.9805e+01
-2.9779e+01
-2.9592e+01
-2.9943e+01
-2.9826e+01
-2.9748e+01
-3.0333e+01
-3.0009e+01
-2.9941e+01
-2.9814e+01
-2.9296e+01
-2.9473e+01
-2.9578e+01
-2.9838e+01
-2.9756e+01
-2.9756e+01
-3.0292e+01
-3.0200e+01
-3.0035e+01
-3.0315e+01
-2.9533e+01
-2.9621e+01
-3.0154e+01
-3.0614e+01
-3.0549e+01
-3.0526e+01
-3.0456e+01
-2.9656e+01
-3.0197e+01
-3.0199e+01
-2.9841e+01
-3.0065e+01
-2.9522e+01
-2.9999e+01
-3.0077e+01
-2.9944e+01
-2.9754e+01
-2.9811e+01
-3.0048e+01
-2.9884e+01
-2.9929e+01
-3.0961e+01
-3.0705e+01
-2.9818e+01
-3.0268e+01
-2.9182e+01
-3.0203e+01
-2.9894e+01
-3.0258e+01
-2.9747e+01
-3.0282e+01
-2.9644e+01
-3.0339e+01
-3.0225e+01
-2.9836e+01
-2.9685e+01
-3.0233e+01
-3.0150e+01
-3.0091e+01
-3.0113e+01
-2.9914e+01
-2.9858e+01
-2.9403e+01
-2.9774e+01
-2.9923e+01
-3.0099e+01
-2.9458e+01
-2.9834e+01
-3.0110e+01
-3.0144e+01
-3.0193e+01
-3.0035e+01
-3.0075e

+01
-2.9630e+01
-3.0431e+01
-2.9819e+01
-2.9684e+01
-3.0360e+01
-3.0110e+01
-2.9754e+01
-3.0237e+01
-2.9694e+01
-2.9949e+01
-3.0301e+01
-2.9957e+01
-2.9800e+01
-2.9792e+01
-3.0356e+01
-2.9645e+01
-3.0057e+01
-2.9908e+01
-2.9752e+01
-2.9955e+01
-3.0490e+01
-2.9401e+01
-3.0021e+01
-2.9773e+01
-2.9680e+01
-3.0583e+01
-2.9735e+01
-3.0017e+01
-2.9538e+01
-3.0146e+01
-3.0238e+01
-3.0060e+01
-3.0123e+01
-2.9664e+01
-2.9977e+01
-2.9982e+01
-3.0151e+01
-3.0680e+01
-3.0062e+01
-3.0260e+01
-2.9948e+01
-2.9751e+01
-3.0545e+01
-2.9969e+01
-3.0112e+01
-2.9602e+01
-3.0102e+01
-3.0024e+01
-2.9531e+01
-2.9929e+01
-2.9805e+01
-2.9855e+01
-2.9750e+01
-2.9933e+01
-2.9883e+01
-3.0055e+01
-3.0543e+01
-2.9315e+01
-2.9949e+01
-2.9968e+01
-3.0132e+01
-3.0229e+01
-2.9979e+01
-2.9956e+01
-2.9270e+01
-3.0332e+01
-3.0472e+01
-2.9533e+01
-2.9497e+01
-3.0472e+01
-3.0422e+01
-2.9579e+01
-3.0227e+01
-3.0197e+01
-3.0336e+01
-3.0578e+01
-2.9592e+01
-3.0244e+01
-3.0033e+01
-2.9809e+01
-3.0173e+01
-3.0032e+01
-2.9635e+01


-3.0385e+01
-3.0239e+01
-2.9893e+01
-3.0042e+01
-3.0430e+01
-2.9893e+01
-3.0311e+01
-3.0423e+01
-2.9687e+01
-3.0262e+01
-2.9926e+01
-3.0124e+01
-2.9567e+01
-3.0445e+01
-3.0053e+01
-2.9929e+01
-2.9665e+01
-2.9872e+01
-3.0189e+01
-3.0028e+01
-3.0016e+01
-2.9726e+01
-2.9593e+01
-3.0580e+01
-2.9137e+01
-3.0401e+01
-2.9869e+01
-3.0288e+01
-3.0352e+01
-2.9749e+01
-3.0060e+01
-2.9867e+01
-2.9306e+01
-3.0287e+01
-2.9629e+01
-3.0298e+01
-2.9768e+01
-2.9821e+01
-2.9820e+01
-2.9670e+01
-3.0382e+01
-2.9923e+01
-2.9720e+01
-2.9538e+01
-3.0286e+01
-2.9351e+01
-3.0103e+01
-3.0375e+01
-2.8992e+01
-2.9865e+01
-2.9337e+01
-2.9670e+01
-3.0111e+01
-3.0028e+01
-2.9999e+01
-3.0224e+01
-3.0362e+01
-3.0011e+01
-3.0074e+01
-2.9730e+01
-3.0347e+01
-3.0268e+01
-2.9719e+01
-3.0075e+01
-2.9931e+01
-2.9737e+01
-2.9083e+01
-3.0134e+01
-2.9712e+01
-2.9565e+01
-2.9656e+01
-2.9682e+01
-3.0245e+01
-3.0135e+01
-2.9614e+01
-2.9875e+01
-2.9327e+01
-2.9994e+01
-3.0285e+01
-2.9973e+01
-2.9372e+01
-2.9756e+01
-2.9388e+01
-3.0

838e+01
-2.9526e+01
-2.9970e+01
-2.9897e+01
-2.9977e+01
-2.9828e+01
-3.0203e+01
-3.0307e+01
-2.9990e+01
-3.0076e+01
-3.0271e+01
-2.9677e+01
-3.0150e+01
-2.9841e+01
-2.9599e+01
-3.0138e+01
-2.9729e+01
-3.0682e+01
-2.9628e+01
-2.9756e+01
-2.9442e+01
-3.0008e+01
-3.0216e+01
-2.9761e+01
-3.0362e+01
-3.0754e+01
-3.0229e+01
-3.0092e+01
-3.0099e+01
-2.9791e+01
-3.0016e+01
-3.0434e+01
-3.0123e+01
-3.0083e+01
-3.0131e+01
-3.0339e+01
-2.9860e+01
-2.9943e+01
-2.9729e+01
-3.0204e+01
-3.0342e+01
-3.0227e+01
-3.0725e+01
-2.9934e+01
-3.0077e+01
-2.9965e+01
-2.9821e+01
-3.0000e+01
-3.0141e+01
-3.0334e+01
-2.9878e+01
-2.9735e+01
-3.0028e+01
-2.9565e+01
-2.9534e+01
-2.9618e+01
-2.9673e+01
-2.9818e+01
-2.9529e+01
-2.9973e+01
-3.0423e+01
-3.0163e+01
-2.9862e+01
-2.9847e+01
-2.9667e+01
-2.9681e+01
-3.0655e+01
-3.0007e+01
-2.9601e+01
-3.0251e+01
-3.0130e+01
-2.9277e+01
-3.0436e+01
-2.9925e+01
-2.9889e+01
-3.0341e+01
-2.9671e+01
-3.0668e+01
-2.9728e+01
-3.0060e+01
-2.9731e+01
-3.0119e+01
-2.9889e+01
-2.9707e

+01
-3.0402e+01
-3.0296e+01
-3.0076e+01
-3.0141e+01
-3.0297e+01
-2.9806e+01
-3.0252e+01
-2.9711e+01
-2.9819e+01
-3.0337e+01
-3.0345e+01
-2.9913e+01
-2.9983e+01
-2.9740e+01
-3.0362e+01
-2.9950e+01
-2.9992e+01
-3.0177e+01
-3.0237e+01
-3.0315e+01
-2.9754e+01
-3.0077e+01
-3.0527e+01
-3.0221e+01
-2.9684e+01
-3.0458e+01
-2.9883e+01
-3.0363e+01
-3.0445e+01
-3.0502e+01
-3.0404e+01
-2.9239e+01
-3.0395e+01
-3.0021e+01
-3.0031e+01
-3.0126e+01
-3.0331e+01
-3.0458e+01
-2.9871e+01
-3.0140e+01
-3.0400e+01
-3.0303e+01
-2.9932e+01
-2.9565e+01
-3.0240e+01
-3.0350e+01
-2.9695e+01
-2.9668e+01
-3.0291e+01
-3.0014e+01
-2.9681e+01
-3.0224e+01
-3.0435e+01
-2.9859e+01
-3.0201e+01
-2.9620e+01
-2.9908e+01
-3.0860e+01
-2.9618e+01
-3.0223e+01
-3.0318e+01
-3.0357e+01
-3.0713e+01
-3.0157e+01
-2.9701e+01
-3.0373e+01
-3.0287e+01
-3.0130e+01
-3.0708e+01
-2.9715e+01
-2.9706e+01
-3.1057e+01
-3.0168e+01
-3.0245e+01
-3.0172e+01
-2.9779e+01
-3.0162e+01
-3.0357e+01
-3.0243e+01
-2.9948e+01
-2.9778e+01
-3.0017e+01
-3.0227e+01
